In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import differential_evolution
from mpl_toolkits.mplot3d import Axes3D
import random
import imageio
import os

In [ ]:
# Define the Rosenbrock functions
def rosenbrock_2d(x):
    return (1 - x[0])**2 + 100 * (x[1] - x[0]**2)**2

def rosenbrock_3d(x):
    return (1 - x[0])**2 + 100 * (x[1] - x[0]**2)**2 + (1 - x[2])**2 + 100 * (x[2] - x[1]**2)**2

In [ ]:
# Genetic Algorithm
def genetic_algorithm(bounds, population_size, generations, mutation_rate, crossover_rate, dimension):
    def initialize_population():
        return [np.random.uniform(low=[b[0] for b in bounds], high=[b[1] for b in bounds]) for _ in range(population_size)]

    def evaluate_population(pop):
        if dimension == 2:
            return [rosenbrock_2d(ind) for ind in pop]
        elif dimension == 3:
            return [rosenbrock_3d(ind) for ind in pop]
        return [rosenbrock_2d(ind) for ind in pop]

    def select_parents(pop, fitness):
        idx = np.argsort(fitness)
        return [pop[i] for i in idx[:2]]  # Select top two individuals

    def crossover(parent1, parent2):
        if random.random() < crossover_rate:
            alpha = random.random()
            child1 = alpha * parent1 + (1 - alpha) * parent2
            child2 = alpha * parent2 + (1 - alpha) * parent1
            return child1, child2
        return parent1, parent2

    def mutate(ind):
        if random.random() < mutation_rate:
            ind += np.random.uniform(-0.5, 0.5, size=dimension)
        return np.clip(ind, [b[0] for b in bounds], [b[1] for b in bounds])  # Keep within bounds

    # Initialize population
    population = initialize_population()
    steps = [population.copy()]

    for gen in range(generations):
        fitness = evaluate_population(population)
        parents = select_parents(population, fitness)

        # Create next generation
        new_population = []
        for _ in range(population_size // 2):
            child1, child2 = crossover(parents[0], parents[1])
            new_population.append(mutate(child1))
            new_population.append(mutate(child2))

        population = new_population
        steps.append(population.copy())

    final_fitness = evaluate_population(population)
    best_idx = np.argmin(final_fitness)
    return population[best_idx], final_fitness[best_idx], steps

In [ ]:
# Generate GIF for 2D Genetic Algorithm
def animate_2d(steps, bounds, filename):
    x = np.linspace(bounds[0][0], bounds[0][1], 400)
    y = np.linspace(bounds[1][0], bounds[1][1], 400)
    X, Y = np.meshgrid(x, y)
    Z = rosenbrock_2d([X, Y])

    images = []
    for i, population in enumerate(steps):
        plt.figure(figsize=(8, 6))
        plt.contourf(X, Y, Z, levels=50, cmap='viridis')
        plt.colorbar(label='Objective Value')
        plt.scatter(*zip(*population), c='red', label='Población', s=20)
        plt.scatter(1,1, c='blue', label='Objetivo', marker="x", s=20)
        plt.title(f"Optimización por algoritmo genético 2D - Generación {i}")
        plt.xlabel('x')
        plt.ylabel('y')
        plt.legend()
        plt.savefig("temp_step_ga.png")
        plt.close()

        images.append(imageio.imread("temp_step_ga.png"))

    imageio.mimsave(filename, images, fps=5, loop=0)

# Set bounds, parameters, and run the genetic algorithm
bounds = [(-2, 2), (-1, 3)]
population_size = 30
generations = 150
mutation_rate = 0.2
crossover_rate = 0.8
dimension = 2

best_solution, best_fitness, steps_ga = genetic_algorithm(bounds, population_size, generations, mutation_rate, crossover_rate, dimension)
print(f"Best Solution: {best_solution}, Fitness: {best_fitness}")

# Create GIF
animate_2d(steps_ga, bounds, "rosenbrock_ga.gif")

In [ ]:
# Generate 3D animation
def animate_3d(steps, bounds, filename):
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Define grid for the Rosenbrock 3D function (projected into two dimensions for visualization)
    x = np.linspace(bounds[0][0], bounds[0][1], 50)
    y = np.linspace(bounds[1][0], bounds[1][1], 50)
    X, Y = np.meshgrid(x, y)
    Z = (1 - X)**2 + 100 * (Y - X**2)**2  # Projection of the function ignoring the third dimension

    images = []
    for i, population in enumerate(steps):
        ax.clear()
        ax.plot_surface(X, Y, Z, cmap='viridis', alpha=0.7, edgecolor='none')

        # Scatter points in 3D space
        for ind in population:
            ax.scatter(ind[0], ind[1], rosenbrock_3d(ind), c='red', s=20)
        ax.scatter(1,1,1, c='blue', label='Objetivo', s=50, marker="x")
        ax.set_title(f"Optimización por algoritmo genético 3D - Generacion {i}")
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        ax.set_zlabel('Objective Value')
        plt.savefig("temp_step_3d_ga.png")
        images.append(imageio.imread("temp_step_3d_ga.png"))

    imageio.mimsave(filename, images, fps=5, loop=0)

# Set bounds, parameters, and run the genetic algorithm
bounds_3d = [(-2, 2), (-1, 3), (-1, 3)]
population_size = 30
generations = 150
mutation_rate = 0.2
crossover_rate = 0.8
dimension = 3

best_solution, best_fitness, steps_ga = genetic_algorithm(bounds_3d, population_size, generations, mutation_rate, crossover_rate, dimension)
print(f"Best Solution: {best_solution}, Fitness: {best_fitness}")

# Create GIF for 3D optimization
animate_3d(steps_ga, bounds_3d, "rosenbrock_3d_ga.gif")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import imageio

# Define the Rastrigin function
def rastrigin(x):
    A = 10
    return A * len(x) + sum([(xi**2 - A * np.cos(2 * np.pi * xi)) for xi in x])

# Genetic Algorithm parameters
POP_SIZE = 50
N_GENERATIONS = 200
DIMENSIONS = 2
BOUNDS = [-5.12, 5.12]
MUTATION_RATE = 0.1
CROSSOVER_RATE = 0.9

# Initialize the population
def initialize_population(size, dimensions, bounds):
    return np.random.uniform(bounds[0], bounds[1], size=(size, dimensions))

# Evaluate the population
def evaluate_population(pop):
    return np.array([rastrigin(ind) for ind in pop])

# Selection (Tournament Selection)
def select(pop, fitness):
    selected = []
    for _ in range(len(pop)):
        i, j = np.random.randint(0, len(pop), 2)
        selected.append(pop[i] if fitness[i] < fitness[j] else pop[j])
    return np.array(selected)

# Crossover (Blend Crossover - BLX-alpha)
def crossover(parent1, parent2, alpha=0.5):
    if np.random.rand() > CROSSOVER_RATE:
        return parent1, parent2
    c_min = np.minimum(parent1, parent2)
    c_max = np.maximum(parent1, parent2)
    diff = c_max - c_min
    child1 = np.random.uniform(c_min - alpha * diff, c_max + alpha * diff)
    child2 = np.random.uniform(c_min - alpha * diff, c_max + alpha * diff)
    return child1, child2

# Mutation
def mutate(ind, bounds):
    if np.random.rand() < MUTATION_RATE:
        idx = np.random.randint(len(ind))
        ind[idx] += np.random.uniform(-1, 1)
        ind[idx] = np.clip(ind[idx], bounds[0], bounds[1])
    return ind

# Optimization process
def genetic_algorithm():
    pop = initialize_population(POP_SIZE, DIMENSIONS, BOUNDS)
    fitness = evaluate_population(pop)
    best_solutions = []
    images = []

    for gen in range(N_GENERATIONS):
        selected = select(pop, fitness)
        next_gen = []

        for i in range(0, len(selected), 2):
            p1, p2 = selected[i], selected[min(i + 1, len(selected) - 1)]
            c1, c2 = crossover(p1, p2)
            next_gen.append(mutate(c1, BOUNDS))
            next_gen.append(mutate(c2, BOUNDS))

        pop = np.array(next_gen[:POP_SIZE])
        fitness = evaluate_population(pop)

        best_idx = np.argmin(fitness)
        best_solutions.append(pop[best_idx])

        # Plot and save frame
        plot_population(pop, fitness, gen, best_solutions[-1])
        plt.savefig(f"frame_{gen}.png")
        images.append(imageio.imread(f"frame_{gen}.png"))
        plt.close()

    # Save GIF
    imageio.mimsave("optimization_process.gif", images, fps=10, loop = 0)
    print("GIF saved as 'optimization_process.gif'.")

# Visualization
def plot_population(pop, fitness, generation, best):
    x = np.linspace(BOUNDS[0], BOUNDS[1], 400)
    y = np.linspace(BOUNDS[0], BOUNDS[1], 400)
    X, Y = np.meshgrid(x, y)
    Z = rastrigin([X, Y])

    plt.figure(figsize=(10, 8))
    plt.contourf(X, Y, Z, levels=50, cmap="viridis")
    plt.colorbar(label="Fitness")
    plt.scatter(pop[:, 0], pop[:, 1], c="red", label="Población")
    plt.scatter(best[0], best[1], c="red", label="Mejor solución")
    plt.scatter(0, 0, c="blue", label="Objetivo")
    plt.title(f"Optimización por algoritmo genético 2D - Generacion {generation}")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.legend()

# Run the optimization
genetic_algorithm()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D  # For 3D plotting

# Adjusted 3D Visualization
def plot_population_3d(pop, fitness, generation, best):
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Create the 3D surface for the Rastrigin function
    x = np.linspace(BOUNDS[0], BOUNDS[1], 100)
    y = np.linspace(BOUNDS[0], BOUNDS[1], 100)
    X, Y = np.meshgrid(x, y)
    Z = rastrigin([X, Y])

    ax.plot_surface(X, Y, Z, cmap='viridis', alpha=0.2, edgecolor='none')
    ax.set_title(f"Optimización por algoritmo genético 3D - Generacion {generation}")
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("Valor de la función")

    # Plot the population
    Z_pop = np.array([rastrigin(ind) for ind in pop])
    ax.scatter(pop[:, 0], pop[:, 1], Z_pop, c="red", label="Población", s=50, depthshade=True)

    # Highlight the best solution
    best_z = rastrigin(best)
    ax.scatter(best[0], best[1], best_z, c="purple", label="Mejor solución", s=100, depthshade=True)

    # Highlight the target point
    best_z = rastrigin(best)
    ax.scatter(0, 0, 0, c="blue", label="Objetivo", s=100, depthshade=True)

    ax.legend()
    plt.savefig(f"frame_{generation}.png")
    plt.close()

# Modify genetic_algorithm to use plot_population_3d
def genetic_algorithm_3d():
    pop = initialize_population(POP_SIZE, DIMENSIONS, BOUNDS)
    fitness = evaluate_population(pop)
    best_solutions = []
    images = []

    for gen in range(N_GENERATIONS):
        selected = select(pop, fitness)
        next_gen = []

        for i in range(0, len(selected), 2):
            p1, p2 = selected[i], selected[min(i + 1, len(selected) - 1)]
            c1, c2 = crossover(p1, p2)
            next_gen.append(mutate(c1, BOUNDS))
            next_gen.append(mutate(c2, BOUNDS))

        pop = np.array(next_gen[:POP_SIZE])
        fitness = evaluate_population(pop)

        best_idx = np.argmin(fitness)
        best_solutions.append(pop[best_idx])

        # Plot and save frame for 3D
        plot_population_3d(pop, fitness, gen, best_solutions[-1])
        images.append(imageio.imread(f"frame_{gen}.png"))

    # Save GIF
    imageio.mimsave("optimization_process_3d.gif", images, fps=10, loop=0)
    print("GIF saved as 'optimization_process_3d.gif'.")

# Run the 3D optimization
genetic_algorithm_3d()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import imageio
import tempfile
import os

# Rosenbrock Function (2D and 3D)
def rosenbrock(x, y, z=None):
    if z is None:
        # 2D Rosenbrock
        return (1 - x)**2 + 100 * (y - x**2)**2
    else:
        # 3D Rosenbrock (extended)
        return (1 - x)**2 + 100 * (y - x**2)**2 + 100 * (z - y**2)**2

# Particle Swarm Optimization (PSO)
class PSO:
    def __init__(self, dim, func, n_particles=50, max_iter=100, w=0.5, c1=1.5, c2=1.5):
        self.dim = dim
        self.func = func
        self.n_particles = n_particles
        self.max_iter = max_iter
        self.w = w   # Inertia weight
        self.c1 = c1 # Cognitive (individual) weight
        self.c2 = c2 # Social (global) weight

        # Initialize particles and velocities
        self.position = np.random.rand(self.n_particles, self.dim) * 10 - 5
        self.velocity = np.random.rand(self.n_particles, self.dim) * 2 - 1
        self.best_position = np.copy(self.position)
        self.best_value = np.array([np.inf] * self.n_particles)

        # Global best solution
        self.global_best_value = np.inf
        self.global_best_position = np.zeros(self.dim)

    def optimize(self):
        gif_images = []
        for t in range(self.max_iter):
            for i in range(self.n_particles):
                value = self.func(self.position[i][0], self.position[i][1]) if self.dim == 2 else self.func(self.position[i][0], self.position[i][1], self.position[i][2])

                # Update personal best
                if value < self.best_value[i]:
                    self.best_value[i] = value
                    self.best_position[i] = self.position[i]

                # Update global best
                if value < self.global_best_value:
                    self.global_best_value = value
                    self.global_best_position = self.position[i]

            # Update velocity and position
            r1 = np.random.rand(self.n_particles, self.dim)
            r2 = np.random.rand(self.n_particles, self.dim)
            cognitive = self.c1 * r1 * (self.best_position - self.position)
            social = self.c2 * r2 * (self.global_best_position - self.position)
            self.velocity = self.w * self.velocity + cognitive + social
            self.position += self.velocity

            # Create an image of the current state for the GIF
            img = self.plot_state(t)
            gif_images.append(img)

        return self.global_best_position, self.global_best_value, gif_images

    def plot_state(self, t):
        fig = plt.figure(figsize=(10, 8))
        if self.dim == 2:
            x = np.linspace(-5, 5, 100)
            y = np.linspace(-5, 5, 100)
            X, Y = np.meshgrid(x, y)
            Z = rosenbrock(X, Y)
            plt.contourf(X, Y, Z, 50, cmap='viridis')
            plt.colorbar()
            plt.scatter(1,1, color='Blue', label='Objetivo', s=50, zorder = 5)
            plt.scatter(self.position[:, 0], self.position[:, 1], color='red', label='Particulas', s=50)
            plt.title(f"Optimización por enjambres en 2D - Iteración {t}")
            plt.xlim(-5, 5)
            plt.ylim(-5, 5)
            plt.legend()
        elif self.dim == 3:
            x = np.linspace(-5, 5, 100)
            y = np.linspace(-5, 5, 100)
            X, Y = np.meshgrid(x, y)
            Z = rosenbrock(X, Y)
            ax = fig.add_subplot(111, projection='3d')
            ax.plot_surface(X, Y, Z, cmap='viridis', edgecolor='none', alpha=0.2)
            ax.scatter(1,1,1, color='Blue', label='Objetivo', s=50)
            ax.scatter(self.position[:, 0], self.position[:, 1], [rosenbrock(x, y) for x, y in zip(self.position[:, 0], self.position[:, 1])],
                       color='red', label='Particulas', s=50)
            ax.set_xlim(-5,5)
            ax.set_ylim(-5,5)
            ax.set_title(f"Optimización por enjambres en 3D - Iteración {t}")
            ax.legend()

        plt.tight_layout()

        # Save the current plot to a temporary file
        with tempfile.NamedTemporaryFile(suffix=".png", delete=False) as tmpfile:
            fig.savefig(tmpfile.name)
            plt.close(fig)
            # Read the temporary file as an image
            img = imageio.imread(tmpfile.name)
            # Delete the temporary file after reading
            os.remove(tmpfile.name)

        return img

# Main function to run PSO and generate GIF
def main():
    dim = 3  # 2D or 3D optimization
    pso = PSO(dim, rosenbrock, n_particles=50, max_iter=100)
    best_pos, best_val, gif_images = pso.optimize()

    print(f"Best Position: {best_pos}")
    print(f"Best Value: {best_val}")

    # Save the GIF
    imageio.mimsave('pso_optimization.gif', gif_images, duration=0.1, loop=0)

if __name__ == "__main__":
    main()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import imageio
import tempfile
import os

# Rastrigin Function (2D and 3D)
def rastrigin(x, y, z=None, A=10):
    if z is None:
        # 2D Rastrigin
        return A * 2 + (x**2 - A * np.cos(2 * np.pi * x)) + (y**2 - A * np.cos(2 * np.pi * y))
    else:
        # 3D Rastrigin
        return A * 3 + (x**2 - A * np.cos(2 * np.pi * x)) + (y**2 - A * np.cos(2 * np.pi * y)) + (z**2 - A * np.cos(2 * np.pi * z))

# Particle Swarm Optimization (PSO)
class PSO:
    def __init__(self, dim, func, n_particles=50, max_iter=100, w=0.5, c1=1.5, c2=1.5):
        self.dim = dim
        self.func = func
        self.n_particles = n_particles
        self.max_iter = max_iter
        self.w = w   # Inertia weight
        self.c1 = c1 # Cognitive (individual) weight
        self.c2 = c2 # Social (global) weight

        # Initialize particles and velocities
        self.position = np.random.rand(self.n_particles, self.dim) * 10 - 5
        self.velocity = np.random.rand(self.n_particles, self.dim) * 2 - 1
        self.best_position = np.copy(self.position)
        self.best_value = np.array([np.inf] * self.n_particles)

        # Global best solution
        self.global_best_value = np.inf
        self.global_best_position = np.zeros(self.dim)

    def optimize(self):
        gif_images = []
        for t in range(self.max_iter):
            for i in range(self.n_particles):
                if self.dim == 2:
                    value = self.func(self.position[i][0], self.position[i][1])
                elif self.dim == 3:
                    value = self.func(self.position[i][0], self.position[i][1], self.position[i][2])

                # Update personal best
                if value < self.best_value[i]:
                    self.best_value[i] = value
                    self.best_position[i] = self.position[i]

                # Update global best
                if value < self.global_best_value:
                    self.global_best_value = value
                    self.global_best_position = self.position[i]

            # Update velocity and position
            r1 = np.random.rand(self.n_particles, self.dim)
            r2 = np.random.rand(self.n_particles, self.dim)
            cognitive = self.c1 * r1 * (self.best_position - self.position)
            social = self.c2 * r2 * (self.global_best_position - self.position)
            self.velocity = self.w * self.velocity + cognitive + social
            self.position += self.velocity

            # Create an image of the current state for the GIF
            img = self.plot_state(t)
            gif_images.append(img)

        return self.global_best_position, self.global_best_value, gif_images

    def plot_state(self, t):
        fig = plt.figure(figsize=(6, 6))
        if self.dim == 2:
            x = np.linspace(-5, 5, 100)
            y = np.linspace(-5, 5, 100)
            X, Y = np.meshgrid(x, y)
            Z = rastrigin(X, Y)
            plt.contourf(X, Y, Z, 50, cmap='viridis')
            plt.colorbar()
            plt.scatter(0,0, color='Blue', label='Objetivo', s=50, zorder = 5)
            plt.scatter(self.position[:, 0], self.position[:, 1], color='red', label='Particulas', s=50)
            plt.title(f"Optimización por enjambres en 2D - Iteración {t}")
            plt.xlim(-5, 5)
            plt.ylim(-5, 5)
            plt.legend()
        elif self.dim == 3:
            x = np.linspace(-5, 5, 100)
            y = np.linspace(-5, 5, 100)
            X, Y = np.meshgrid(x, y)
            Z = rastrigin(X, Y)
            ax = fig.add_subplot(111, projection='3d')
            ax.plot_surface(X, Y, Z, cmap='viridis', edgecolor='none', alpha=0.2)
            ax.scatter(0,0,0, color='Blue', label='Objetivo', s=50)
            ax.scatter(self.position[:, 0], self.position[:, 1], [rosenbrock(x, y) for x, y in zip(self.position[:, 0], self.position[:, 1])],
                       color='red', label='Particulas', s=50)
            ax.set_xlim(-5,5)
            ax.set_ylim(-5,5)
            ax.set_zlim(0,80)
            ax.set_title(f"Optimización por enjambres en 3D - Iteración {t}")
            ax.legend()

        plt.tight_layout()

        # Save the current plot to a temporary file
        with tempfile.NamedTemporaryFile(suffix=".png", delete=False) as tmpfile:
            fig.savefig(tmpfile.name)
            plt.close(fig)
            # Read the temporary file as an image
            img = imageio.imread(tmpfile.name)
            # Delete the temporary file after reading
            os.remove(tmpfile.name)

        return img

# Main function to run PSO and generate GIF
def main():
    dim = 3  # 2D or 3D optimization
    pso = PSO(dim, rastrigin, n_particles=50, max_iter=100)
    best_pos, best_val, gif_images = pso.optimize()

    print(f"Best Position: {best_pos}")
    print(f"Best Value: {best_val}")

    # Save the GIF
    imageio.mimsave('pso_rastrigin_optimization.gif', gif_images, duration=0.1, loop=0)

if __name__ == "__main__":
    main()


In [ ]:
import numpy as np
import scipy.optimize as opt
import imageio
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Rosenbrock function in 2D and 3D
def rosenbrock(x):
    if len(x) == 2:
        return (1 - x[0])**2 + 100 * (x[1] - x[0]**2)**2
    elif len(x) == 3:
        return (1 - x[0])**2 + 100 * (x[1] - x[0]**2)**2 + 100 * (x[2] - x[1]**2)**2

# Differential Evolution optimization with trajectory tracking
def optimize_rosenbrock(bounds):
    trajectory = []

    # Callback function to store the trajectory
    def callback(x, convergence):
        trajectory.append(np.copy(x))

    result = opt.differential_evolution(
        rosenbrock,
        bounds=bounds,
        strategy='best1bin',
        maxiter=100,
        callback=callback
    )
    return result, trajectory

# Plot 2D Rosenbrock function with optimization process
def plot_rosenbrock_2d(trajectory, gif_filename="rosenbrock_2d_optimization.gif"):
    x = np.linspace(-2, 2, 400)
    y = np.linspace(-2, 2, 400)
    X, Y = np.meshgrid(x, y)
    Z = (1 - X)**2 + 100 * (Y - X**2)**2
    i = 0

    images = []
    fig, ax = plt.subplots()
    ax.contourf(X, Y, Z, 50, cmap='viridis')
    ax.set_xlim(-2,2)
    ax.set_ylim(-2,2)
    ax.set_xlabel('x')
    ax.set_ylabel('y')

    for point in trajectory:
        ax.set_title(f"Optimización por Evolución Diferencial en 2D - Iteración {i}")
        ax.plot(point[0], point[1], 'ro', markersize=5)
        ax.scatter(1,1, color = 'blue')
        plt.draw()
        filename = f"temp_2d_{len(images)}.png"
        fig.savefig(filename)
        images.append(imageio.imread(filename))
        i += 1

    imageio.mimsave(gif_filename, images, duration=0.1,loop=0)
    plt.close(fig)

# Plot 3D Rosenbrock function with optimization process
def plot_rosenbrock_3d(trajectory, gif_filename="rosenbrock_3d_optimization.gif"):
    x = np.linspace(-2, 2, 50)
    y = np.linspace(-2, 2, 50)
    X, Y = np.meshgrid(x, y)
    Z = (1 - X)**2 + 100 * (Y - X**2)**2  # Fixed z-slice for visualization
    i = 0

    images = []
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(X, Y, Z, cmap='viridis', alpha=0.3)
    ax.set_title("Optimization Process in 3D")
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')

    for point in trajectory:
        ax.cla()  # Clear the plot for each frame
        ax.plot_surface(X, Y, Z, cmap='viridis', alpha=0.3)
        ax.scatter(point[0], point[1], rosenbrock(point), color='r', s=50)
        ax.scatter(1, 1, 1, color='blue', s=50)
        ax.set_title(f"Optimización por Evolución Diferencial en 3D - Iteración {i}")
        ax.set_xlim(-2,2)
        ax.set_ylim(-2,2)
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        ax.set_zlabel('z')


        plt.draw()
        filename = f"temp_3d_{len(images)}.png"
        fig.savefig(filename)
        images.append(imageio.imread(filename))
        i += 1

    imageio.mimsave(gif_filename, images, duration=0.1, loop=0)
    plt.close(fig)

# Main execution
def main():
    # Optimize and visualize 2D Rosenbrock
    bounds_2d = [(-2, 2), (-2, 2)]
    result_2d, trajectory_2d = optimize_rosenbrock(bounds_2d)
    print(f"2D Optimization Result: {result_2d.x} with value: {result_2d.fun}")
    plot_rosenbrock_2d(trajectory_2d)

    # Optimize and visualize 3D Rosenbrock
    bounds_3d = [(-2, 2), (-2, 2), (-2, 2)]
    result_3d, trajectory_3d = optimize_rosenbrock(bounds_3d)
    print(f"3D Optimization Result: {result_3d.x} with value: {result_3d.fun}")
    plot_rosenbrock_3d(trajectory_3d)

if __name__ == "__main__":
    main()

In [ ]:
import numpy as np
import scipy.optimize as opt
import imageio
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Rastrigin function in 2D and 3D
def rastrigin(x):
    A = 10
    n = len(x)
    return A * n + sum([(xi**2 - A * np.cos(2 * np.pi * xi)) for xi in x])

# Differential Evolution optimization with trajectory tracking
def optimize_rastrigin(bounds):
    trajectory = []

    # Callback function to store the trajectory
    def callback(x, convergence):
        trajectory.append(np.copy(x))

    result = opt.differential_evolution(
        rastrigin,
        bounds=bounds,
        strategy='best1bin',
        maxiter=100,
        callback=callback
    )
    return result, trajectory

# Plot 2D Rastrigin function with optimization process
def plot_rastrigin_2d(trajectory, gif_filename="rastrigin_2d_optimization.gif"):
    x = np.linspace(-5.12, 5.12, 400)
    y = np.linspace(-5.12, 5.12, 400)
    X, Y = np.meshgrid(x, y)
    Z = 10 * 2 + (X**2 - 10 * np.cos(2 * np.pi * X)) + (Y**2 - 10 * np.cos(2 * np.pi * Y))
    i = 0

    images = []
    fig, ax = plt.subplots()
    ax.contourf(X, Y, Z, 50, cmap='viridis')
    ax.set_xlim(-5.12,5.12)
    ax.set_ylim(-5.12,5.12)
    ax.set_xlabel('x')
    ax.set_ylabel('y')

    for point in trajectory:
        ax.set_title(f"Optimización por Evolución Diferencial en 2D - Iteración {i}")
        ax.plot(point[0], point[1], 'ro', markersize=5)
        ax.scatter(0,0, color = 'blue')
        plt.draw()
        filename = f"temp_2d_{len(images)}.png"
        fig.savefig(filename)
        images.append(imageio.imread(filename))
        i += 1

    imageio.mimsave(gif_filename, images, duration=0.1, loop=0)
    plt.close(fig)

# Plot 3D Rastrigin function with optimization process
def plot_rastrigin_3d(trajectory, gif_filename="rastrigin_3d_optimization.gif"):
    x = np.linspace(-5.12, 5.12, 400)
    y = np.linspace(-5.12, 5.12, 400)
    X, Y = np.meshgrid(x, y)
    Z = 10 * 2 + (X**2 - 10 * np.cos(2 * np.pi * X)) + (Y**2 - 10 * np.cos(2 * np.pi * Y))
    i = 0

    images = []
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(X, Y, Z, cmap='viridis', alpha=0.3)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')

    for point in trajectory:
        ax.cla()  # Clear the plot for each frame
        ax.set_title(f"Optimización por Evolución Diferencial en 3D - Iteración {i}")
        ax.plot_surface(X, Y, Z, cmap='viridis', alpha=0.3)
        ax.scatter(point[0], point[1], rastrigin(point), color='r', s=50)
        ax.scatter(0, 0, 0, color='blue', s=50)
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        ax.set_zlabel('z')
        i += 1

        plt.draw()
        filename = f"temp_3d_{len(images)}.png"
        fig.savefig(filename)
        images.append(imageio.imread(filename))

    imageio.mimsave(gif_filename, images, duration=0.1, loop=0)
    plt.close(fig)

# Main execution
def main():
    # Optimize and visualize 2D Rastrigin
    bounds_2d = [(-5.12, 5.12), (-5.12, 5.12)]
    result_2d, trajectory_2d = optimize_rastrigin(bounds_2d)
    print(f"2D Optimization Result: {result_2d.x} with value: {result_2d.fun}")
    plot_rastrigin_2d(trajectory_2d)

    # Optimize and visualize 3D Rastrigin
    bounds_3d = [(-5.12, 5.12), (-5.12, 5.12), (-5.12, 5.12)]
    result_3d, trajectory_3d = optimize_rastrigin(bounds_3d)
    print(f"3D Optimization Result: {result_3d.x} with value: {result_3d.fun}")
    plot_rastrigin_3d(trajectory_3d)

if __name__ == "__main__":
    main()
